In [ ]:
!wget http://download.tensorflow.org/models/object_detection/mask_rcnn_resnet50_atrous_coco_2018_01_28.tar.gz

In [ ]:
!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!sudo python3 -m pip install -e detectron2_repo

In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

In [ ]:
import ipywidgets.widgets as widgets

controller = widgets.Controller(index=0)
display(controller)

In [1]:
import traitlets
from xarm.wrapper import XArmAPI


class xArm(traitlets.HasTraits):    
    servo1 = traitlets.Float()
    servo2 = traitlets.Float()
    servo3 = traitlets.Float()
    servo4 = traitlets.Float()
    servo5 = traitlets.Float()
    
    @traitlets.validate('servo1')
    def _clip_servo1(self, proposal):
        if proposal['value'] < -50.0:
            print(f"Received servo1 value of {proposal['value']}. Value not allowed below -50.0. Returning -50.0.")
            return -50.0
        elif proposal['value'] > 50.0:
            print(f"Received servo1 value of {proposal['value']}. Value not allowed to exceed 50.0. Returning 50.0.")
            return 50.0
        else:
            return proposal['value']
    
    @traitlets.validate('servo2')
    def _clip_servo2(self, proposal):
        """
        Clip values for servo2 on the xArm to safe values for our work surface
        """
        if proposal['value'] < -40:
            print(f"Received servo2 value of {proposal['value']}. Value not allowed below -40.0. Returning 0.0.")
            return -40.0
        
        elif proposal['value'] > 15.0:
            print(f"Received servo2 value of {proposal['value']}. Value not allowed to exceed 15.0. Returning 15.0.")
        else:
            return proposal['value']
        
    @traitlets.validate('servo3')
    def _clip_servo3(self, proposal):
        """
        Clip values for servo3 on the xArm to safe values for our work surface
        """
        if proposal['value'] < 0.0:
            print(f"Received servo3 value of {proposal['value']}. Value not allowed below 0.0. Returning 0.0.")
            return 0.0
        
        elif proposal['value'] > 180.0:
            print(f"Received servo3 value of {proposal['value']}. Value not allowed to exceed -180.0. Returning -180.0.")
        else:
            return proposal['value']
        
    @traitlets.validate('servo4')
    def _clip_servo4(self, proposal):
        """
        Clip values for servo4 on the xArm to safe values for our work surface
        """
        if proposal['value'] > -20.0:
            print(f"Received servo4 value of {proposal['value']}. Value not allowed to exceed -20.0. Returning -20.0.")
            return -20.0
        
        elif proposal['value'] < -90.0:
            print(f"Received servo value of {proposal['value']}. Value not allowed below -90.0. Returning -90.0.")
        else:
            return proposal['value']
        
    @traitlets.validate('servo5')
    def _clip_servo5(self, proposal):
        """
        Clip values for servo5 on the xArm to safe values for our work surface
        """
        if proposal['value'] > 180.0:
            print(f"Received sensor5 value of {proposal['value']}. Value not allowed to exceed 180.0. Returning 180.0.")
            return 180.0
        elif proposal['value'] < -180.0:
            print(f"Received sensor5 value of {proposal['value']}. Value not allowed below -180.0. Returning -180.0")
        else:
            return proposal['value']

class DrinkArm(xArm):    
    def __init__(self, ip: str, motion_enable: bool,
                 mode: int, state: int, speed: int, *args, **kwargs):
        super(DrinkArm, self).__init__(*args, **kwargs)
        self.ip = ip
        self.motion_enable = motion_enable
        self.mode = mode
        self.state = state
        self.speed = speed
        
        self.arm = XArmAPI(self.ip)
        self.arm.motion_enable(enable=motion_enable)
        self.arm.set_mode(mode)
        self.arm.set_state(state=state)
        
        @traitlets.observe('servo1')
        def _on_servo1(self, change):
            self.arm.set_servo_angle(servo_id=1, angle=change['new'], speed=self.speed, wait=False)
        
        @traitlets.observe('servo2')
        def _on_servo2(self, change):
            self.arm.set_servo_angle(servo_id=2, angle=change['new'], speed=self.speed, wait=False)
        
        @traitlets.observe('servo3')
        def _on_servo3(self, change):
            self.arm.set_servo_angle(servo_id=3, angle=change['new'], speed=self.speed, wait=False)
        
        @traitlets.observe('servo4')
        def _on_servo4(self, change):
            self.arm.set_servo_angle(servo_id=4, angle=change['new'], speed=self.speed, wait=False)
        
        @traitlets.observe('servo5')
        def _on_servo5(self, change):
            self.arm.set_servo_angle(servo_id=5, angle=change['new'], speed=self.speed, wait=False)

In [2]:
arm = DrinkArm(ip='192.168.1.244', motion_enable=True, mode=0,
               state=0, speed=50)

main-socket connect 192.168.1.244 success
report-socket connect 192.168.1.244 success
is_old_protocol: False
version_number: 1.3.1
=============sync_all
[motion_enable], xArm is not ready to move
[set_state], xArm is ready to move


In [13]:
def update_servo1(change):
    print(change['new'])

arm.observe(update_servo1, 'value')

In [14]:
arm.servo1 = 10